In [16]:
import pandas
import numpy as np

In [17]:
def replace_missing_values(df):
    #replace missing values in categorical/binary variables with mode
    #replace missing values in non-categorical variables with median

    #column indexes for categorical variables
    cat_indexes=[3,5,6,23,24,25,26,27,28,29,30,31,32,33]
    #column indexes for binary variables
    bin_indexes=[7,17,18,19,53,54,55,56,57]
    #column indexes for non-categorical variables
    nor_indexes=[2,4,15,16,20,21,22,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52]
    
    for index in range (2,58):
        name = list(df.iloc[:,[index]])[0]
        if index in nor_indexes:
            df[name] = df[name].fillna(df[name].median(axis=0))
        else:
            df[name] = df[name].fillna(df[name].mode()[0])
    return df

In [18]:
df = pandas.read_csv('train.csv')
#convert -1 value to NaN
df.replace(to_replace=[-1],value=[np.nan], inplace = True)
if df.isnull().sum().sum() != 0:
    df = replace_missing_values(df)

In [19]:
df_test = pandas.read_csv('test.csv')
df_test.replace(to_replace=[-1],value=[np.nan], inplace = True)
idx = 1
new_col = np.zeros(df_test.shape[0]) 
df_test.insert(loc=idx, column='target', value=new_col)
if df_test.isnull().sum().sum() != 0:
    #if there are NaN values in the data frame, proceed to replace the NaN values with median/mode
    df_test = replace_missing_values(df_test)

In [20]:
X = df.iloc[:,2:58]
y = df.iloc[:,1]

In [21]:
from sklearn.model_selection import train_test_split

features_train, features_test, lables_train, labels_test = \
    train_test_split(X, y, test_size=0.33, random_state=42)

In [22]:
# checking
print len(features_train)
print len(features_test)

398792
196420


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

def evaluate(labels_test, pred):
    acc = accuracy_score(labels_test, pred)
    ps = precision_score(labels_test, pred)
    rs = recall_score(labels_test, pred)

    print "acc:", acc
    print "precision_score:", ps
    print "recall_score:", rs

In [24]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV

#params = {'min_samples_split':[6,7,8,9,10], 'criterion':('gini', 'entropy')}
clf_tree = tree.DecisionTreeClassifier(min_samples_split=10, criterion="gini")
#clf = GridSearchCV(clf_tree, params)

clf_tree.fit(features_train, lables_train)

#print "parameters:", clf.best_params_

pred = clf_tree.predict(features_test)

evaluate(labels_test, pred)
print "\n"

acc: 0.935062620914
precision_score: 0.0435640309058
recall_score: 0.0367953346293




In [26]:
from sklearn.naive_bayes import GaussianNB

clf_nb = GaussianNB()

clf_nb.fit(features_train, lables_train)

pred = clf_nb.predict(features_test)

evaluate(labels_test, pred)
print "\n"

acc: 0.9014662458
precision_score: 0.069200226886
recall_score: 0.135517911691


